Algoritmos Metaheuristicos  
Tarea 3 - TSP  
Autor: Ing. Daniel Eduardo González Alvarado  
Profesor: Dr. Marco Aceves

# Importar Librerias

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Travelling Salesman Problem (TSP)

El problema del viajero es uno de los problemas más famosos en el campo de la optimizacion computacional. El enunciado clasico es:  

"Sean N ciudades de un territorio. El objetivo es encontrar una ruta que, comenzando y terminando en una ciudad en concreto pase solo una vez por cada una de las ciudades y minimice la distancia recorrida por el viajero."

Es decir, encontrar una permutacion:

$P = \{C_0,C_1,\dots,C_N-1\}$ tal que $d_p = \sum_{i=0}^{N-1} d(c_i,c_{i+1})$ sea minima.

## Ciudades

In [2]:
cities = [
("Mexico City",0 ,19.4204503765706, -99.14569125006386),     # Mexico City 
("Quito",1,-0.22504624887149816, -78.49988938997475),        # Quito
("Miami",2,25.762008625108713, -80.19106117773524),          # Miami
("San Salvador",3,13.698816559559196, -89.19139082077601),   # San Salvador
("Mendoza",4,-32.890254485684615, -68.8454728365394),        # Mendoza
("Guadalajara",5,20.674513177689914, -103.33814458178117),   # Guadalajara
("Merida",6,20.969848004507575, -89.59865182312967),         # Merida
("Washington D.C.",7,38.90971457165538, -77.02266752763703), # Washington D.C.
("Monterrey",8,25.68095070483323, -100.30007435921115),      # Monterrey
("Managua",9,12.110415225874924, -86.22824820040704),        # Managua
("Caracas",10,10.475819042313416, -66.90959192246824),       # Caracas
("Boston",11,42.35091277211663, -71.05104866254476),         # Boston
("Buenos Aires",12,-34.609506741669584, -58.39281746089803), # Buenos Aires
("New York",13,40.71067551341166, -74.01246498137415),       # New York
("Panama City",14,8.942473452575438, -79.17169679439931),    # Panama City
("Brasilia",15,-15.664864237549638, -46.621579947655576),    # Brasilia
("Montevideo",16,-34.90728943056367, -56.18638267328233),    # Montevideo
("Bogota",17,4.708521026911784, -74.0679815641729)]          # Bogota

df = pd.DataFrame(cities, columns=["city","id" ,"lat", "lon"])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      18 non-null     int64  
 1   city    18 non-null     object 
 2   lat     18 non-null     float64
 3   lon     18 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 708.0+ bytes


In [6]:
#############################################################################

# Orden descendente geografico
orden = [11, 13, 7, 2, 8, 5, 0, 6, 3, 9, 14, 10, 17, 1, 15, 16, 12, 4]

# Reordenar según lista
df_ordenado = df.set_index("id").loc[orden].reset_index()

print(df_ordenado)
df = df_ordenado

    id             city        lat         lon
0   11           Boston  42.350913  -71.051049
1   13         New York  40.710676  -74.012465
2    7  Washington D.C.  38.909715  -77.022668
3    2            Miami  25.762009  -80.191061
4    8        Monterrey  25.680951 -100.300074
5    5      Guadalajara  20.674513 -103.338145
6    0      Mexico City  19.420450  -99.145691
7    6           Merida  20.969848  -89.598652
8    3     San Salvador  13.698817  -89.191391
9    9          Managua  12.110415  -86.228248
10  14      Panama City   8.942473  -79.171697
11  10          Caracas  10.475819  -66.909592
12  17           Bogota   4.708521  -74.067982
13   1            Quito  -0.225046  -78.499889
14  15         Brasilia -15.664864  -46.621580
15  16       Montevideo -34.907289  -56.186383
16  12     Buenos Aires -34.609507  -58.392817
17   4          Mendoza -32.890254  -68.845473


In [ ]:
def new_route():
    return random.sample(range(18), 18)

# print(new_route())

[13, 6, 10, 9, 14, 1, 3, 15, 2, 11, 8, 17, 5, 12, 16, 0, 4, 7]


In [7]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattergeo(
    lon=df["lon"],
    lat=df["lat"],
    text=df["id"],
    mode="markers+text",
    textposition="bottom center"

))
# Recortado a nuestra zona de Interes
fig.update_geos(
    visible=False, resolution=110,
    showcountries=True, countrycolor="Black",
    lonaxis=dict(range=[-130, -30]),   # limita la longitud (Oeste hasta Este)
    lataxis=dict(range=[-55, 50])      # limita la latitud (Sur a Norte)
)
# Tamaño Grafica
fig.update_layout(
    width=500, height=500,
    margin=dict(r=0, t=0, l=0, b=0)
)
# Captira de Mapa
fig.show(config={
    "staticPlot": True,       
    "displayModeBar": False
})

In [8]:
# Indice de Ciudades (0,...,17)
n=len(df) 
route = list(range(n))

# Trazar Ruta
lon_route = df.iloc[route]["lon"].tolist()
lat_route = df.iloc[route]["lat"].tolist()
lon_route += [lon_route[0]]
lat_route += [lat_route[0]]

fig.add_trace(go.Scattergeo(
    lon=lon_route,
    lat=lat_route,
    mode="lines",
    line=dict(width=2),
    hoverinfo="skip",
    name="Ruta"
))

fig.show(config={
    "staticPlot": True,       # congela el gráfico (sin pan/zoom/selección)
    "displayModeBar": False
})

# Generar Población

Tomando en cueta que nuestra Poblacion total son las combinaciones de las 18 ciudades tenemos que:  
El total seria $18! = 6,402,373,705,728,000$ posibles combinaciones

En la Imagen anterior podemos ver la ruta trazada con las ciudades en el orden en que fueron escritas por lo tanto si revisamos la ruta veremos lo siguiente:

In [7]:
print(route)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


Es importante tomar en cuenta que inicia en la posicion 0 pero termina en la 17 esto es esperado a este momento sin embargo para nuestro problema tenemos que tener en cuenta que el lugar de inicio y final debe ser la misma ciudad por lo que debemos agregar esta consideracion.

In [32]:
def back_home (route):
    # Comprobar que no sea mayor a 19 (0 -> 17) + 1
    while len(route) > 18: 
        route.pop()

    start = route[0]
    route.append(start)
    return route

In [ ]:
def back_home_idx(route):
    # route_idx es lista de 18 enteros
    if len(route) != 18:
        raise ValueError(f"La ruta tiene {len(route)} nodos; se esperaban 18.")
    return route + [route[0]]

In [37]:
r1= new_route()
r1 = back_home_idx(r1)
r1

[15, 16, 2, 3, 12, 0, 11, 13, 7, 1, 4, 9, 17, 5, 8, 6, 10, 14, 15]

In [9]:
r1 = back_home(route)
print(r1)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 0]


In [17]:
def haversine_matrix(lat, lon):
    R = 6371008.8  # radio medio Tierra (m)
    phi = np.radians(lat)[:, None]     # (N,1)
    lam = np.radians(lon)[:, None]     # (N,1)
    dphi = phi.T - phi                 # (1,N) - (N,1) -> (N,N)
    dlam = lam.T - lam
    a = np.sin(dphi/2)**2 + np.cos(phi) * np.cos(phi.T) * np.sin(dlam/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c  # (N,N) en metros

In [18]:
lat = df["lat"].to_numpy(float)
lon = df["lon"].to_numpy(float)
D = haversine_matrix(lat, lon)

In [19]:
print(D)

[[      0.         3136783.02517061 2067095.16289206 1236525.05296006
  6648170.1854239   459576.23167521 1010928.49066113 3034822.61242734
   706143.50602092 1602407.48650551 3596037.684795   3667299.52622476
  7391650.94203159 3360687.31295295 2444577.48704656 6942713.76025051
  7553433.9151211  3171796.60165781]
 [3136783.02517061       0.         2895331.2753105  1945200.54610748
  3770269.21763955 3562879.67430623 2647515.56782912 4354202.70638463
  3711443.73095196 1615255.47467142 1748878.09718127 4792674.68334735
  4357237.28416676 4574399.08995664 1022094.87875745 3897006.2780232
  4498418.83406324  737078.7084796 ]
 [2067095.16289206 2895331.2753105        0.         1637663.03945859
  6631519.35690432 2428448.43733922 1097695.33327525 1491668.04124754
  2012518.99362995 1644646.06412086 2200658.04437409 2024666.90388886
  7095185.74816127 1757493.98258487 1873347.12279648 5866509.4743525
  7204080.75868806 2430237.14667966]
 [1236525.05296006 1945200.54610748 1637663.0394585

In [ ]:
def generar_poblacion(num):
    generacion = []
    for _ in range(num):
        ind = new_route()
        ind = back_home(ind)
        generacion.append(ind)
        print(ind)
    return generacion

gen = generar_poblacion(3)

KeyError: 0

# Revisar
-Distancia Haversine  
-Vicenty